In [1]:
import sys
sys.path.append("..")

from functions import get_data, data_split_train_val

import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
# Load Fashion-MNIST CSV (test set)
X, y = get_data("../data/fashion-mnist_test.csv")

X_train, X_val, y_train, y_val = data_split_train_val(X, y)

print("Train:", X_train.shape)
print("Val:", X_val.shape)

Train: (8000, 784)
Val: (2000, 784)


In [3]:
# convert numpy arrays to PyTorch tensors
X_train_tensor = torch.from_numpy(X_train).float()
y_train_tensor = torch.from_numpy(y_train).long()

X_val_tensor = torch.from_numpy(X_val).float()
y_val_tensor = torch.from_numpy(y_val).long()

In [4]:
print("Before reshape:")
print("X_train_tensor shape:", X_train_tensor.shape)
print("X_val_tensor shape:", X_val_tensor.shape)

Before reshape:
X_train_tensor shape: torch.Size([8000, 784])
X_val_tensor shape: torch.Size([2000, 784])


In [5]:
num_train = X_train_tensor.shape[0]
num_val = X_val_tensor.shape[0]

print(num_train, num_val)

8000 2000


In [6]:
X_train_tensor = X_train_tensor.reshape(num_train, 1, 28, 28)
X_val_tensor = X_val_tensor.reshape(num_val, 1, 28, 28)
print("X_train_tensor reshape:", X_train_tensor.shape)
print("X_val_tensor reshape:", X_val_tensor.shape)

X_train_tensor reshape: torch.Size([8000, 1, 28, 28])
X_val_tensor reshape: torch.Size([2000, 1, 28, 28])


In [7]:
class CNN_LeakyReLU(nn.Module):
    def __init__(self):
        super().__init__()
        self.act = nn.LeakyReLU(0.01)

        self.conv1 = nn.Conv2d(1, 8, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(8, 16, 3)

        self.fc1 = nn.Linear(16*5*5, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(self.act(self.conv1(x)))
        x = self.pool(self.act(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = self.act(self.fc1(x))
        return self.fc2(x)

In [15]:
data = CNN_LeakyReLU()   # create the model

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(data.parameters(), lr=0.01)

epochs = 20

for epoch in range(epochs):
    optimizer.zero_grad()
    
    outputs = data(X_train_tensor)         
    loss = criterion(outputs, y_train_tensor)
    
    loss.backward()
    optimizer.step()
    
    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}")

Epoch 1/20, Loss: 2.3044
Epoch 2/20, Loss: 2.2714
Epoch 3/20, Loss: 2.1526
Epoch 4/20, Loss: 1.9300
Epoch 5/20, Loss: 1.6043
Epoch 6/20, Loss: 1.2853
Epoch 7/20, Loss: 1.2120
Epoch 8/20, Loss: 1.4322
Epoch 9/20, Loss: 1.4647
Epoch 10/20, Loss: 1.4049
Epoch 11/20, Loss: 1.1411
Epoch 12/20, Loss: 1.1985
Epoch 13/20, Loss: 1.1410
Epoch 14/20, Loss: 1.0357
Epoch 15/20, Loss: 0.9911
Epoch 16/20, Loss: 0.9996
Epoch 17/20, Loss: 1.0106
Epoch 18/20, Loss: 0.9918
Epoch 19/20, Loss: 0.9494
Epoch 20/20, Loss: 0.9062


In [16]:
with torch.no_grad():
    outputs = data(X_val_tensor)
    predictions = torch.argmax(outputs, 1)
    correct = predictions == y_val_tensor
    correct_float = correct.float()
    accuracy = correct_float.mean().item()
    
print("Validation accuracy:", accuracy)

Validation accuracy: 0.6990000009536743
